In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
data = pd.read_csv("./gender.csv", encoding='latin1')

In [3]:
import re

In [4]:


def normalize_text(s):
    # just in case
    s = str(s)
    s = s.lower()
    
    # remove punctuation that is not word-internal (e.g., hyphens, apostrophes)
    s = re.sub('\s\W',' ',s)
    s = re.sub('\W\s',' ',s)
    
    # make sure we didn't introduce any double spaces
    s = re.sub('\s+',' ',s)
    
    return s

data['text_norm'] = [normalize_text(s) for s in data['text']]
data['description_norm'] = [normalize_text(s) for s in data['description']]



In [5]:
# pull the data into vectors
# function to split the data for cross-validation
from sklearn.model_selection import train_test_split
# function for transforming documents into counts
from sklearn.feature_extraction.text import CountVectorizer
# function for encoding categories
from sklearn.preprocessing import LabelEncoder
vectorizer = CountVectorizer()
x = vectorizer.fit_transform(data['text_norm'])
le= LabelEncoder()

data['gender'] = data['gender'].factorize()[0]
data_gender = le.fit_transform(data.gender)
y = data_gender

In [6]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)
n_nodes_hl1 = 500 
n_nodes_hl3 = 500 
n_nodes_hl2 = 500 

n_classes = 10

batch_size = 100
print(x_train.shape,y_train.shape)
lengh = x_train.shape[0]
leng = x_train.shape[1]
print(lengh,leng)
x = tf.placeholder('float',[lengh, None])
y = tf.placeholder('float')

def neural_network_model(data):
    hidden_1_layer = {'weights':tf.Variable(tf.random_normal([lengh,n_nodes_hl1])),
    'biases':tf.Variable(tf.random_normal([n_nodes_hl1]))}
    hidden_2_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl1,n_nodes_hl2])),
    'biases':tf.Variable(tf.random_normal([n_nodes_hl2]))}
    hidden_3_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl2,n_nodes_hl3])),
    'biases':tf.Variable(tf.random_normal([n_nodes_hl3]))}
    output_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl3,n_classes])),
    'biases':tf.Variable(tf.random_normal([n_classes]))}

    l1 = tf.add(tf.matmul(data,hidden_1_layer['weights']) ,hidden_1_layer['biases'])
    l1 = tf.nn.relu(l1)

    l2 = tf.add(tf.matmul(l1,hidden_2_layer['weights']) ,hidden_2_layer['biases'])
    l2 = tf.nn.relu(l2)

    l3 = tf.add(tf.matmul(l2,hidden_3_layer['weights'])  ,hidden_3_layer['biases'])
    l3 = tf.nn.relu(l3)

    output = tf.matmul(l3,output_layer['weights']) + output_layer['biases']

    return output

def train_neural_network(x):
    prediction = neural_network_model(x)
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = prediction,labels = y))

    optimizer = tf.train.AdamOptimizer().minimize(cost)

    hm_epochs = 10

    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())

        for epoch in range(hm_epochs):
            epoch_loss = 0
            i=0
            while i<lengh:
                start = i
                end = i + batch_size
                batch_x = np.array(x_train[start:end],dtype=object)
                batch_y = np.array(y_train[start:end],dtype=object)
     
                _,c=sess.run([optimizer, cost], feed_dict={x: batch_x, y: batch_y })
                epoch_loss += c
                i += batch_size
            print('Epoch',epoch,'complete out of',hm_epochs,'loss:',epoch_loss)




            correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y,1))
            accuracy = tf.reduce_mean(tf.cast(correct,'float'))
            print('Accuracy',accuracy.eval({x:x_test,y:y_test}))

train_neural_network(x)



(16040, 43121) (16040,)
16040 43121


ValueError: setting an array element with a sequence.